# Multi-Head Attention
The last part of attention. This is going to be a lot of code. Brace yourselves.

![Brace yourself](./img/brace.jpg)

This is going to be the last notebook (for a while) that involves attention. :) Again, the code here has been adapted from Sebastian Raschka's Build an LLM From Scratch book. And as usual, I start with a `torch` implementation, and move to implementing with `ttnn`.

At this point, I think we are getting more and more comfortable with using `ttnn`, so the amount of detail when it comes to porting code over will be less granular.

## Import Dependent Modules

Import `torch` and set the seed to `123` for consistency.

In [46]:
import torch
from torch import nn

torch.manual_seed(123)

We will also need some sample inputs. We'll bring back the `context` found in the previous notebook involving causal attention.

Our inputs have to also be transformed into a single batch of 2. So actually we have a dimension of (2, 6, 3) for our inputs. (We get the second input by just doing a stack -- resulting in repetition)

Note that for the assignment for `d_out`, we will be sticking to the book by making it 1 less than `d_in` to just check answers.

In [47]:
context = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

batch = torch.stack((context, context), dim=0)
print(batch.shape)

d = context.shape
d_in = context.shape[1]
d_out = context.shape[1] - 1

d_in, d_out

torch.Size([2, 6, 3])


(3, 2)

## Define CausalAttention

We'll bring back the `CausalAttention` class here to demonstrate that at the most basic level, a `MultiHeadAttention` is just a stack of `CausalAttention` instances. 

When we perform a forward pass through a `MultiheadAttention` module, we are just iterating through each `CausalAttention` instance and feeding the inputs through one after another.

In [48]:

class CausalAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
    super().__init__()

    self.d_out = d_out
    
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    
    self.dropout = nn.Dropout(dropout)

    self.mask = torch.triu(
      torch.ones(context_length, context_length),
      diagonal=1
    )

  def forward(self, x): 
    b, num_tokens, d_in = x.shape
    keys = self.W_query(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    # transpose the last 2 dimensions while leaving the batch dimension alone.
    attn_scores = queries @ keys.transpose(1, 2)
    attn_scores.masked_fill(
      self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
    )
    attn_weights = torch.softmax(
      attn_scores / keys.shape[-1]**0.5, dim=-1
    )

    attn_weights = self.dropout(attn_weights)

    context_vec = attn_weights @ values

    return context_vec


Let's test `CausalAttention` by passing in the batch.

In [49]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
ca(batch)


tensor([[[-0.5287, -0.0976],
         [-0.5293, -0.1053],
         [-0.5293, -0.1052],
         [-0.5287, -0.1072],
         [-0.5287, -0.1038],
         [-0.5288, -0.1080]],

        [[-0.5287, -0.0976],
         [-0.5293, -0.1053],
         [-0.5293, -0.1052],
         [-0.5287, -0.1072],
         [-0.5287, -0.1038],
         [-0.5288, -0.1080]]], grad_fn=<UnsafeViewBackward0>)

## MultiHeadAttention With Wrapped CausalAttention

`torch.cat` is used to concatenate the outputs of each head along the last dimension.

The number of interest is the the `num_heads` argument to the `MultiHeadAttentionWrapper` constructor. This will tell us how many `CausalAttention` instances we must initialize. 

When performing the forward pass, we'll perform the operation and concatenate the results of a `CausalAttention` instance for each head given the batch input.

So if we initially have a 3x2 tensor, with 2 heads we will get 3x4. With 3 heads, it will become 3x6, etc.

In [50]:
class MultiHeadAttentionWrapper(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()

    self.heads = nn.ModuleList(
      [
        CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
        for _ in range(num_heads)
      ]
    )

  def forward(self, x):
    return torch.cat([head(x) for head in self.heads], dim=-1)



Let's test. For good safety measure, set the seed to 123 again.

In [51]:
torch.manual_seed(123)
context_length = batch.shape[1]
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
context_vecs, context_vecs.shape

(tensor([[[-0.5287, -0.0976,  0.5122,  0.3448],
          [-0.5293, -0.1053,  0.5123,  0.3449],
          [-0.5293, -0.1052,  0.5121,  0.3448],
          [-0.5287, -0.1072,  0.5096,  0.3438],
          [-0.5287, -0.1038,  0.5078,  0.3427],
          [-0.5288, -0.1080,  0.5113,  0.3446]],
 
         [[-0.5287, -0.0976,  0.5122,  0.3448],
          [-0.5293, -0.1053,  0.5123,  0.3449],
          [-0.5293, -0.1052,  0.5121,  0.3448],
          [-0.5287, -0.1072,  0.5096,  0.3438],
          [-0.5287, -0.1038,  0.5078,  0.3427],
          [-0.5288, -0.1080,  0.5113,  0.3446]]], grad_fn=<CatBackward0>),
 torch.Size([2, 6, 4]))

## MultiHeadAttention in Torch Optimized

Now, as stated in the book, we can perform a more optimized forward operation on the batch in one go. This means that we're not using our `CausalAttention` class anymore. In fact, we will build those operations into the forward method. 

If you read the code, you will notice that the code is very similar to `CausalAttention`.

`view` is used to reshape the dimensions with the provided.

In [52]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()

    self.d_out = d_out
    self.num_heads = num_heads
    self.head_dim = d_out // num_heads # Reduce the projection dimension to match the output dim

    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    self.out_proj = nn.Linear(d_out, d_out)
    self.dropout = nn.Dropout(dropout)

    self.mask = torch.triu(
      torch.ones(context_length, context_length),
      diagonal=1
    )

  def forward(self, x):
    b, num_tokens, d_in = x.shape

    keys = self.W_key(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    keys = keys.view(
      b,
      num_tokens,
      self.num_heads,
      self.head_dim
    )
    values = values.view(
      b,
      num_tokens,
      self.num_heads,
      self.head_dim
    )
    queries = queries.view(
      b,
      num_tokens,
      self.num_heads,
      self.head_dim
    )
    
    keys = keys.transpose(1, 2)
    queries = queries.transpose(1, 2)
    values = values.transpose(1, 2)
    
    attn_scores = queries @ keys.transpose(2, 3)
    mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

    attn_scores.masked_fill_(mask_bool, -torch.inf)

    attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
    attn_weights = self.dropout(attn_weights)

    context_vec = torch.transpose((attn_weights @ values), 1, 2)
    context_vec = context_vec.contiguous().view(
      b,
      num_tokens,
      self.d_out
    )

    context_vec = self.out_proj(context_vec)

    return context_vec

We will initialize the `MultiHeadAttention` class to have `num_heads` to be `2`. At this point we should be pretty close to the result that we want assuming that we make sure our seed is `123` :) 

In [53]:
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2 
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

context_vecs, context_vecs.shape


(tensor([[[0.3190, 0.4858],
          [0.2943, 0.3897],
          [0.2856, 0.3593],
          [0.2693, 0.3873],
          [0.2639, 0.3928],
          [0.2575, 0.4028]],
 
         [[0.3190, 0.4858],
          [0.2943, 0.3897],
          [0.2856, 0.3593],
          [0.2693, 0.3873],
          [0.2639, 0.3928],
          [0.2575, 0.4028]]], grad_fn=<ViewBackward0>),
 torch.Size([2, 6, 2]))

Save the result so that we can compare later.

In [54]:
context_vecs_torch = context_vecs
context_vecs_torch

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)

## TTNN Example

In [55]:
import ttnn

torch.manual_seed(123)

In [56]:
core_grid_y = 8
core_grid_x = 8

MINUS_INFINITY=-1e9

In [57]:
class MultiHeadAttention_ttnn(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, device, qkv_bias=False):
    super().__init__()

    self.device = device
    self.dropout_prob = dropout

    self.d_out = d_out
    self.num_heads = num_heads
    self.head_dim = d_out // num_heads # Reduce the projection dimension to match the output dim

    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.out_proj = nn.Linear(d_out, d_out)
    self.dropout = nn.Dropout(dropout)
    self.mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)

    self.W_query_ttnn = ttnn.from_torch(
      self.W_query.weight,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )
    self.W_key_ttnn = ttnn.from_torch(
      self.W_key.weight,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )
    self.W_value_ttnn = ttnn.from_torch(
      self.W_value.weight,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )
    self.out_proj_ttnn = ttnn.from_torch(
      self.out_proj.weight,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )
    self.mask_ttnn = ttnn.from_torch(
      self.mask,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )

  def forward(self, x):
    b, num_tokens, d_in = x.shape

    x_ttnn = ttnn.from_torch(
      x,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device
    )

    keys_ttnn = ttnn.linear(
      x_ttnn,
      self.W_key_ttnn,
      transpose_b=True,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )
    queries_ttnn = ttnn.linear(
      x_ttnn,
      self.W_query_ttnn,
      transpose_b=True,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )
    values_ttnn = ttnn.linear(
      x_ttnn,
      self.W_value_ttnn,
      transpose_b=True,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )
    
    keys_ttnn = ttnn.reshape(keys_ttnn, (b, num_tokens, self.num_heads, self.head_dim))
    values_ttnn = ttnn.reshape(values_ttnn, (b, num_tokens, self.num_heads, self.head_dim))
    queries_ttnn = ttnn.reshape(queries_ttnn, (b, num_tokens, self.num_heads, self.head_dim))

    # NOTE! This is intentional. We want the transposed version of keys_ttnn. That's why the
    # shape has a different permutation than the values_ttnn and queries_ttnn!
    keys_transposed_ttnn = ttnn.permute(keys_ttnn, (0, 2, 3, 1))
    values_ttnn = ttnn.permute(values_ttnn, (0, 2, 1, 3))
    queries_ttnn = ttnn.permute(queries_ttnn, (0, 2, 1, 3))

    attn_scores_ttnn = ttnn.matmul(
      queries_ttnn, 
      keys_transposed_ttnn,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )

    attn_scores_ttnn = attn_scores_ttnn * (1 / (self.head_dim ** 0.5))

    attn_mask_ttnn = self.mask_ttnn[:num_tokens, :num_tokens] * MINUS_INFINITY
    attn_mask_ttnn = ttnn.reshape(attn_mask_ttnn, (1, 1, num_tokens, num_tokens))
    attn_scores_ttnn += attn_mask_ttnn
    
    attn_weights_ttnn = ttnn.softmax(attn_scores_ttnn, dim=-1)

    if self.dropout_prob > 0.0:
      attn_weights_ttnn = ttnn.experimental.dropout(
        attn_weights_ttnn,
        seed=123,
        probability=self.dropout_prob,
        scale=1.0 / (1.0 - self.dropout_prob)
      )

    
    context_vec_ttnn = ttnn.matmul(
      attn_weights_ttnn,
      values_ttnn,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    ) 

    context_vec_ttnn = ttnn.permute(context_vec_ttnn, (0, 2, 1, 3))
    context_vec_ttnn = ttnn.reshape(context_vec_ttnn, (b, num_tokens, self.d_out))

    out_proj_bias_ttnn = ttnn.from_torch(
      self.out_proj.bias,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device
    )

    context_vec_ttnn = ttnn.linear(
      context_vec_ttnn,
      self.out_proj_ttnn,
      transpose_b=True,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )
    context_vec_ttnn = ttnn.add(context_vec_ttnn, out_proj_bias_ttnn)

    # Send the context vector back to the CPU
    context_vec = ttnn.from_device(context_vec_ttnn)
    context_vec = ttnn.to_torch(context_vec)

    return context_vec

Let's test. We should be getting a similar result.

In [58]:

torch.manual_seed(123)

device_id = 0
device = ttnn.open_device(device_id=device_id)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention_ttnn(d_in, d_out, context_length, 0.0, num_heads=2, device=device)

context_vecs = mha(batch)

ttnn.close_device(device)

context_vecs, context_vecs.shape

                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0


(TorchTensor([[[0.3145, 0.4922],
               [0.2891, 0.4043],
               [0.2773, 0.3828],
               [0.2656, 0.4004],
               [0.2598, 0.4062],
               [0.2520, 0.4199]],
 
              [[0.3145, 0.4922],
               [0.2891, 0.4043],
               [0.2773, 0.3828],
               [0.2656, 0.4004],
               [0.2598, 0.4062],
               [0.2520, 0.4199]]], dtype=torch.bfloat16),
 torch.Size([2, 6, 2]))

In [59]:
context_vecs_ttnn = context_vecs
context_vecs_ttnn

TorchTensor([[[0.3145, 0.4922],
              [0.2891, 0.4043],
              [0.2773, 0.3828],
              [0.2656, 0.4004],
              [0.2598, 0.4062],
              [0.2520, 0.4199]],

             [[0.3145, 0.4922],
              [0.2891, 0.4043],
              [0.2773, 0.3828],
              [0.2656, 0.4004],
              [0.2598, 0.4062],
              [0.2520, 0.4199]]], dtype=torch.bfloat16)

## Results Comparison
Let's compare our results so that we can see whether or not the `ttnn` results compares well with the `torch` comparison. 

It is expected that the numbers are slightly different, but we are expecting this and willing to accept some level of tolerance.

In [60]:
from scripts.compare_tensors import compare_tensors

torch_output = context_vecs_torch   
ttnn_output = context_vecs_ttnn    

results = compare_tensors(
    torch_output,
    ttnn_output,
    max_diff_tolerance=0.025
)

=== Tensor Comparison ===
Shapes: PyTorch torch.Size([2, 6, 2]), TTNN torch.Size([2, 6, 2])
Data types: PyTorch torch.bfloat16, TTNN torch.bfloat16

Tolerance Checks:
  Max Absolute Diff: 0.023438 (Tolerance: 0.025000) ✅ PASS
  Mean Absolute Diff: 0.009949 (Tolerance: 0.020000) ✅ PASS
  Correlation: 0.996094 (Tolerance: 0.990000) ✅ PASS

Overall Status: ✅ PASS

Sample Value Comparisons (first 3 positions):
  Position [0,0,0]: PyTorch=0.318359, TTNN=0.314453, Diff=0.003906 ✅
  Position [0,0,1]: PyTorch=0.486328, TTNN=0.492188, Diff=0.005859 ✅
  Position [0,1,0]: PyTorch=0.294922, TTNN=0.289062, Diff=0.005859 ✅
  Position [0,1,1]: PyTorch=0.390625, TTNN=0.404297, Diff=0.013672 ✅
  Position [0,2,0]: PyTorch=0.285156, TTNN=0.277344, Diff=0.007812 ✅
  Position [0,2,1]: PyTorch=0.359375, TTNN=0.382812, Diff=0.023438 ✅
  Position [1,0,0]: PyTorch=0.318359, TTNN=0.314453, Diff=0.003906 ✅
  Position [1,0,1]: PyTorch=0.486328, TTNN=0.492188, Diff=0.005859 ✅
  Position [1,1,0]: PyTorch=0.294922, 